<a href="https://colab.research.google.com/github/ykitaguchi77/deepVOG/blob/master/Running_deepVOG_in_colab2.1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Running Github codes in Google Colab



In [8]:
#Install requirements
!pip install scikit-video
!pip install scikit-image
!pip install tensorflow==1.14.0
import keras

#Google driveをマウント

In [9]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#__init__py

In [0]:
import numpy as np
import matplotlib as mpl
from matplotlib.patches import Ellipse
from skimage.measure import label, regionprops
from skimage.morphology import closing, square
from skimage.color import rgb2gray
from skimage.transform import resize
from skimage.draw import ellipse_perimeter, line, circle_perimeter, line_aa
import skvideo.io as skv
import pdb
import os
import warnings
import traceback
import json

In [0]:
#bwperim.py
# vim: set ts=4 sts=4 sw=4 expandtab smartindent:
#
# This file was originally part of the octave-forge project
# Ported to python by Luis Pedro Coelho <luis@luispedro.org> (February 2008)
# Copyright (C) 2006       Soren Hauberg
# Copyright (C) 2008-2010  Luis Pedro Coelho (Python port)
# 
# This program is free software; you can redistribute it and/or modify
# it under the terms of the GNU General Public License as published by
# the Free Software Foundation; either version 2, or (at your option)
# any later version.
# 
# This program is distributed in the hope that it will be useful, but
# WITHOUT ANY WARRANTY; without even the implied warranty of
# MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the GNU
# General Public License for more details. 
# 
# You should have received a copy of the GNU General Public License
# along with this file.  If not, see <http://www.gnu.org/licenses/>.

def bwperim(bw, n=4):
    """
    perim = bwperim(bw, n=4)
    Find the perimeter of objects in binary images.
    A pixel is part of an object perimeter if its value is one and there
    is at least one zero-valued pixel in its neighborhood.
    By default the neighborhood of a pixel is 4 nearest pixels, but
    if `n` is set to 8 the 8 nearest pixels will be considered.
    Parameters
    ----------
      bw : A black-and-white image
      n : Connectivity. Must be 4 or 8 (default: 8)
    Returns
    -------
      perim : A boolean image
    """

    if n not in (4,8):
        raise ValueError('mahotas.bwperim: n must be 4 or 8')
    rows,cols = bw.shape

    # Translate image by one pixel in all directions
    north = np.zeros((rows,cols))
    south = np.zeros((rows,cols))
    west = np.zeros((rows,cols))
    east = np.zeros((rows,cols))

    north[:-1,:] = bw[1:,:]
    south[1:,:]  = bw[:-1,:]
    west[:,:-1]  = bw[:,1:]
    east[:,1:]   = bw[:,:-1]
    idx = (north == bw) & \
          (south == bw) & \
          (west  == bw) & \
          (east  == bw)
    if n == 8:
        north_east = np.zeros((rows, cols))
        north_west = np.zeros((rows, cols))
        south_east = np.zeros((rows, cols))
        south_west = np.zeros((rows, cols))
        north_east[:-1, 1:]   = bw[1:, :-1]
        north_west[:-1, :-1] = bw[1:, 1:]
        south_east[1:, 1:]     = bw[:-1, :-1]
        south_west[1:, :-1]   = bw[:-1, 1:]
        idx &= (north_east == bw) & \
               (south_east == bw) & \
               (south_west == bw) & \
               (north_west == bw)
    return ~idx * bw

In [0]:
#draw_ellipse.py
def isolate_islands(prediction, threshold):
    bw = closing(prediction > threshold , square(3))
    labelled = label(bw)  
    regions_properties = regionprops(labelled)
    max_region_area = 0
    select_region = 0
    for region in regions_properties:
        if region.area > max_region_area:
            max_region_area = region.area
            select_region = region
    output = np.zeros(labelled.shape)
    if select_region == 0:
        return output
    else:
        output[labelled == select_region.label] = 1
        return output

# input: output from bwperim -- 2D image with perimeter of the ellipse = 1
def gen_ellipse_contour_perim(perim, color = "r"): 
    # Vertices
    input_points = np.where(perim == 1)
    if (np.unique(input_points[0]).shape[0]) < 6 or (np.unique(input_points[1]).shape[0]< 6) :
        return None
    else:
        try:
            vertices = np.array([input_points[0], input_points[1]]).T
            # Contour
            fitted = LSqEllipse()
            fitted.fit([vertices[:,1], vertices[:,0]])
            center, w,h, radian = fitted.parameters()
            ell = mpl.patches.Ellipse(xy = [center[0],center[1]], width = w*2, height = h*2, angle = np.rad2deg(radian), fill = False, color = color)
            # Because of the np indexing of y-axis, orientation needs to be minus
            rr, cc = ellipse_perimeter(int(np.round(center[0])), int(np.round(center[1])), int(np.round(w)), int(np.round(h)), -radian)
            return (rr, cc, center, w, h, radian, ell)
        except:
            return None

def gen_ellipse_contour_perim_compact(perim): 
    # Vertices
    input_points = np.where(perim == 1)
    if (np.unique(input_points[0]).shape[0]) < 6 or (np.unique(input_points[1]).shape[0]< 6) :
        return None
    else:
        try:
            vertices = np.array([input_points[0], input_points[1]]).T
            # Contour
            fitted = LSqEllipse()
            fitted.fit([vertices[:,1], vertices[:,0]])
            center, w,h, radian = fitted.parameters()
            # Because of the np indexing of y-axis, orientation needs to be minus
            return (center, w,h, radian)
        except:
            return None

def fit_ellipse(img, threshold = 0.5, color = "r", mask=None):

    isolated_pred = isolate_islands(img, threshold = threshold)
    perim_pred = bwperim(isolated_pred)

    # masking eyelid away from bwperim_output. Currently not available in DeepVOG (But will be used in DeepVOG-3D)
    if mask is not None:
        mask_bool = mask < 0.5
        perim_pred[mask_bool] = 0

    # masking bwperim_output on the img boundaries as 0 
    perim_pred[0, :] = 0
    perim_pred[perim_pred.shape[0]-1, :] = 0
    perim_pred[:, 0] = 0
    perim_pred[:, perim_pred.shape[1]-1] = 0
    ellipse_info = gen_ellipse_contour_perim(perim_pred, color)

    return ellipse_info

def fit_ellipse_compact(img, threshold = 0.5, mask=None):
    """Fitting an ellipse to the thresholded pixels which form the largest connected area.
    Args:
        img (2D numpy array): Prediction from the DeepVOG network (240, 320), float [0,1]
        threshold (scalar): thresholding pixels for fitting an ellipse
        mask (2D numpy array): Prediction from DeepVOG-3D network for eyelid region (240, 320), float [0,1].
                                intended for masking away the eyelid such as the fitting is better
    Returns:
        ellipse_info (tuple): A tuple of (center, w, h, radian), center is a list [x-coordinate, y-coordinate] of the ellipse centre. 
                                None is returned if no ellipse can be found.
    """
    isolated_pred = isolate_islands(img, threshold = threshold)
    perim_pred = bwperim(isolated_pred)

    # masking eyelid away from bwperim_output. Currently not available in DeepVOG (But will be used in DeepVOG-3D)
    if mask is not None:
        mask_bool = mask < 0.5
        perim_pred[mask_bool] = 0

    # masking bwperim_output on the img boundaries as 0 
    perim_pred[0, :] = 0
    perim_pred[perim_pred.shape[0]-1, :] = 0
    perim_pred[:, 0] = 0
    perim_pred[:, perim_pred.shape[1]-1] = 0
    
    ellipse_info = gen_ellipse_contour_perim_compact(perim_pred)
    return ellipse_info

In [0]:
#ellipses.py
"""Demonstration of least-squares fitting of ellipses
    __author__ = "Ben Hammel, Nick Sullivan-Molina"
    __credits__ = ["Ben Hammel", "Nick Sullivan-Molina"]
    __maintainer__ = "Ben Hammel"
    __email__ = "bdhammel@gmail.com"
    __status__ = "Development"
    Requirements 
    ------------
    Python 2.X or 3.X
    numpy
    matplotlib
    References
    ----------
    (*) Halir, R., Flusser, J.: 'Numerically Stable Direct Least Squares 
        Fitting of Ellipses'
    (**) http://mathworld.wolfram.com/Ellipse.html
    (***) White, A. McHale, B. 'Faraday rotation data analysis with least-squares 
        elliptical fitting'
"""

class LSqEllipse:

    def fit(self, data):
        """Lest Squares fitting algorithm 
        Theory taken from (*)
        Solving equation Sa=lCa. with a = |a b c d f g> and a1 = |a b c> 
            a2 = |d f g>
        Args
        ----
        data (list:list:float): list of two lists containing the x and y data of the
            ellipse. of the form [[x1, x2, ..., xi],[y1, y2, ..., yi]]
        Returns
        ------
        coef (list): list of the coefficients describing an ellipse
           [a,b,c,d,f,g] corresponding to ax**2+2bxy+cy**2+2dx+2fy+g
        """
        x, y = numpy.asarray(data, dtype=float)

        #Quadratic part of design matrix [eqn. 15] from (*)
        D1 = numpy.mat(numpy.vstack([x**2, x*y, y**2])).T
        #Linear part of design matrix [eqn. 16] from (*)
        D2 = numpy.mat(numpy.vstack([x, y, numpy.ones(len(x))])).T
        
        #forming scatter matrix [eqn. 17] from (*)
        S1 = D1.T*D1
        S2 = D1.T*D2
        S3 = D2.T*D2  
        
        #Constraint matrix [eqn. 18]
        C1 = numpy.mat('0. 0. 2.; 0. -1. 0.; 2. 0. 0.')

        #Reduced scatter matrix [eqn. 29]
        M=C1.I*(S1-S2*S3.I*S2.T)

        #M*|a b c >=l|a b c >. Find eigenvalues and eigenvectors from this equation [eqn. 28]
        eval, evec = numpy.linalg.eig(M) 

        # eigenvector must meet constraint 4ac - b^2 to be valid.
        cond = 4*numpy.multiply(evec[0, :], evec[2, :]) - numpy.power(evec[1, :], 2)
        a1 = evec[:, numpy.nonzero(cond.A > 0)[1]]
        
        #|d f g> = -S3^(-1)*S2^(T)*|a b c> [eqn. 24]
        a2 = -S3.I*S2.T*a1
        
        # eigenvectors |a b c d f g> 
        self.coef = numpy.vstack([a1, a2])
        self._save_parameters()
            
    def _save_parameters(self):
        """finds the important parameters of the fitted ellipse
        
        Theory taken form http://mathworld.wolfram
        Args
        -----
        coef (list): list of the coefficients describing an ellipse
           [a,b,c,d,f,g] corresponding to ax**2+2bxy+cy**2+2dx+2fy+g
        Returns
        _______
        center (List): of the form [x0, y0]
        width (float): major axis 
        height (float): minor axis
        phi (float): rotation of major axis form the x-axis in radians 
        """

        #eigenvectors are the coefficients of an ellipse in general form
        #a*x^2 + 2*b*x*y + c*y^2 + 2*d*x + 2*f*y + g = 0 [eqn. 15) from (**) or (***)
        a = self.coef[0,0]
        b = self.coef[1,0]/2.
        c = self.coef[2,0]
        d = self.coef[3,0]/2.
        f = self.coef[4,0]/2.
        g = self.coef[5,0]
        
        #finding center of ellipse [eqn.19 and 20] from (**)
        x0 = (c*d-b*f)/(b**2.-a*c)
        y0 = (a*f-b*d)/(b**2.-a*c)
        
        #Find the semi-axes lengths [eqn. 21 and 22] from (**)
        numerator = 2*(a*f*f+c*d*d+g*b*b-2*b*d*f-a*c*g)
        denominator1 = (b*b-a*c)*( (c-a)*numpy.sqrt(1+4*b*b/((a-c)*(a-c)))-(c+a))
        denominator2 = (b*b-a*c)*( (a-c)*numpy.sqrt(1+4*b*b/((a-c)*(a-c)))-(c+a))
        width = numpy.sqrt(numerator/denominator1)
        height = numpy.sqrt(numerator/denominator2)

        # angle of counterclockwise rotation of major-axis of ellipse to x-axis [eqn. 23] from (**)
        # or [eqn. 26] from (***).
        phi = .5*numpy.arctan((2.*b)/(a-c))

        self._center = [x0, y0]
        self._width = width
        self._height = height
        self._phi = phi

    @property
    def center(self):
        return self._center

    @property
    def width(self):
        return self._width

    @property
    def height(self):
        return self._height

    @property
    def phi(self):
        """angle of counterclockwise rotation of major-axis of ellipse to x-axis 
        [eqn. 23] from (**)
        """
        return self._phi

    def parameters(self):
        return self._center, self._width, self._height, self._phi


def make_test_ellipse(center=[1,1], width=1, height=.6, phi=3.14/5):
    """Generate Elliptical data with noise
    
    Args
    ----
    center (list:float): (<x_location>, <y_location>)
    width (float): semimajor axis. Horizontal dimension of the ellipse (**)
    height (float): semiminor axis. Vertical dimension of the ellipse (**)
    phi (float:radians): tilt of the ellipse, the angle the semimajor axis
        makes with the x-axis 
    Returns
    -------
    data (list:list:float): list of two lists containing the x and y data of the
        ellipse. of the form [[x1, x2, ..., xi],[y1, y2, ..., yi]]
    """
    t = numpy.linspace(0, 2*numpy.pi, 1000)
    x_noise, y_noise = numpy.random.rand(2, len(t))
    
    ellipse_x = center[0] + width*numpy.cos(t)*numpy.cos(phi)-height*numpy.sin(t)*numpy.sin(phi) + x_noise/2.
    ellipse_y = center[1] + width*numpy.cos(t)*numpy.sin(phi)+height*numpy.sin(t)*numpy.cos(phi) + y_noise/2.

    return [ellipse_x, ellipse_y]

In [0]:
#eyefitter.py
"""
Unless specified, all units are in pixels. 
All calculations are in camera frame (conversion would be commented)
"""


class SingleEyeFitter(object):

    def __init__(self, focal_length, pupil_radius, initial_eye_z, image_shape=(240, 320)):
        self.focal_length = focal_length
        self.image_shape = image_shape

        self.pupil_radius = pupil_radius
        self.vertex = [0, 0, -focal_length]
        self.initial_eye_z = initial_eye_z

        # (p,n) of unprojected gaze vector and pupil 3D position in SINGLE OBSERVATION
        self.current_gaze_pos = 0  # reserved for (3,1) np.array in camera frame
        self.current_gaze_neg = 0  # reserved for (3,1) np.array in camera frame
        self.current_pupil_3Dcentre_pos = 0  # reserved for (3,1) np.array in camera frame
        self.current_pupil_3Dcentre_neg = 0  # reserved for (3,1) np.array in camera frame
        self.current_ellipse_centre = 0  # reserved for numpy array (2,1) in numpy indexing frame

        # List of parameters across a number (m) of observations
        self.unprojected_gaze_vectors = []  # A list: ["gaze_positive"~np(m,3), "gaze_negative"~np(m,3)]
        self.unprojected_3D_pupil_positions = []  # [ "pupil_3Dcentre_positive"~np(m,3), "pupil_3Dcentre_negative"~np(m,3) ]
        self.ellipse_centres = None  # reserved for numpy array (m,2) in numpy indexing frame,
        # m = number of fitted ellipse centres corresponding to the projected gaze lines
        self.selected_gazes = None  # reserved for (m,3) np.array in camera frame
        self.selected_pupil_positions = None  # reserved for (m,3) np.array in camera frame

        # Parameters of the eye model for consistent pupil estimate after initialisation
        self.projected_eye_centre = None  # reserved for numpy array (2,1). Centre coordinate in numpy indexing frame.
        self.eye_centre = None  # reserved for (3,1) numpy array. 3D centre coordinate in camera frame
        self.aver_eye_radius = None  # Scaler

        # Results of consistent pupil estimate
        self.pupil_new_position_max = None  # numpy array (3,1)
        self.pupil_new_position_min = None  # numpy array (3,1)
        self.pupil_new_radius_max = None  # scalar
        self.pupil_new_radius_min = None  # scalar
        self.pupil_new_gaze_max = None  # numpy array (3,1)
        self.pupil_new_gaze_min = None  # numpy array (3,1)

    def unproject_single_observation(self, prediction, mask=None, threshold=0.5):
        # "prediction" is an numpy array with shape (image_height, image_width) and data type: float [0-1]
        # Our deeplearning model's outpupt is Y~(240, 320, 3),
        # you will have to slice it manually as Y[:,:,1] as input to this function.
        try:
            assert len(prediction.shape) == 2
            assert prediction.shape == self.image_shape
        except(AssertionError):
            raise AssertionError(
                "Shape of the observation input has to be (image_height, image_width) specified in the initialization of object, or if default, (240,320)")
        # Fit an ellipse from the prediction map
        ellipse_info = fit_ellipse(prediction, mask=mask)
        rr, cc, centre, w, h, radian = None, None, None, None, None, None
        ellipse_confidence = 0

        # We unproject the gaze vectors and pupil centre only if an ellipse has been detected
        if ellipse_info is not None:

            (rr, cc, centre, w, h, radian, ell) = ellipse_info
            ellipse_confidence = computeEllipseConfidence(prediction, centre, w, h, radian)

            # Convert centre coordinates from numpy indexing frame to camera frames
            centre_cam = centre.copy()
            centre_cam[0] = centre_cam[0] - self.image_shape[1] / 2
            centre_cam[1] = centre_cam[1] - self.image_shape[0] / 2

            # Convert ellipse parameters to the coefficients of the general form of ellipse equation
            A, B, C, D, E, F = convert_ell_to_general(centre_cam[0], centre_cam[1], w, h, radian)
            ell_co = (A, B, C, D, E, F)

            # Unproject the ellipse to obtain 2 ambiguous gaze vectors with numpy shape (3,1),
            # and pupil_centre with numpy shape (3,1)
            unprojected_gaze_pos, unprojected_gaze_neg, unprojected_pupil_3Dcentre_pos, unprojected_pupil_3Dcentre_neg = unprojectGazePositions(
                self.vertex, ell_co, self.pupil_radius)

            # Normalize the gaze vectors and only take their real component
            unprojected_gaze_pos = unprojected_gaze_pos / np.linalg.norm(unprojected_gaze_pos)
            unprojected_gaze_neg = unprojected_gaze_neg / np.linalg.norm(unprojected_gaze_neg)

            unprojected_gaze_pos, unprojected_gaze_neg, unprojected_pupil_3Dcentre_pos, unprojected_pupil_3Dcentre_neg = np.real(
                unprojected_gaze_pos), np.real(unprojected_gaze_neg), np.real(unprojected_pupil_3Dcentre_pos), np.real(
                unprojected_pupil_3Dcentre_neg)
            self.current_gaze_pos, self.current_gaze_neg, self.current_pupil_3Dcentre_pos, self.current_pupil_3Dcentre_neg = unprojected_gaze_pos, unprojected_gaze_neg, unprojected_pupil_3Dcentre_pos, unprojected_pupil_3Dcentre_neg
            self.current_ellipse_centre = np.array(centre).reshape(2, 1)
        else:
            self.current_gaze_pos, self.current_gaze_neg, self.current_pupil_3Dcentre_pos, self.current_pupil_3Dcentre_neg = None, None, None, None
            self.current_ellipse_centre = None

        return self.current_gaze_pos, self.current_gaze_neg, self.current_pupil_3Dcentre_pos, self.current_pupil_3Dcentre_neg, (
        rr, cc, centre, w, h, radian, ellipse_confidence)

    def add_to_fitting(self):
        # Append parameterised gaze lines for fitting
        if (self.current_gaze_pos is None) or (self.current_gaze_neg is None) or (
                self.current_pupil_3Dcentre_pos is None) or (self.current_pupil_3Dcentre_neg is None) or (
                self.current_ellipse_centre is None):
            raise TypeError(
                'No ellipse was caught in this observation, thus "None" is being added for fitting set, which is not allowed. Please manually skip this condition.')

        # Store the gaze vectors and pupil 3D centres
        if (len(self.unprojected_gaze_vectors) == 0) or (len(self.unprojected_3D_pupil_positions) == 0) or (
                self.ellipse_centres is None):
            self.unprojected_gaze_vectors.append(self.current_gaze_pos.reshape(1, 3))
            self.unprojected_gaze_vectors.append(self.current_gaze_neg.reshape(1, 3))
            self.unprojected_3D_pupil_positions.append(self.current_pupil_3Dcentre_pos.reshape(1, 3))
            self.unprojected_3D_pupil_positions.append(self.current_pupil_3Dcentre_neg.reshape(1, 3))
            self.ellipse_centres = self.current_ellipse_centre.reshape(1, 2)
        else:
            self.unprojected_gaze_vectors[0] = np.vstack(
                (self.unprojected_gaze_vectors[0], self.current_gaze_pos.reshape(1, 3)))
            self.unprojected_gaze_vectors[1] = np.vstack(
                (self.unprojected_gaze_vectors[1], self.current_gaze_neg.reshape(1, 3)))
            self.unprojected_3D_pupil_positions[0] = np.vstack(
                (self.unprojected_3D_pupil_positions[0], self.current_pupil_3Dcentre_pos.reshape(1, 3)))
            self.unprojected_3D_pupil_positions[1] = np.vstack(
                (self.unprojected_3D_pupil_positions[1], self.current_pupil_3Dcentre_neg.reshape(1, 3)))
            self.ellipse_centres = np.vstack((self.ellipse_centres, self.current_ellipse_centre.reshape(1, 2)))

    def fit_projected_eye_centre(self, ransac=False, max_iters=1000, min_distance=2000):
        # You will need to determine when to fit outside of the class
        if (self.unprojected_gaze_vectors is None) or (self.ellipse_centres is None):
            msg = "No unprojected gaze lines or ellipse centres were found (not yet initalized). It is likely that the network fails to segment the pupil from the video. Please ensure your input video contains only a single eye but not other facial/body features."
            raise TypeError(msg)

        # Combining positive and negative gaze vectors
        a = np.vstack((self.ellipse_centres, self.ellipse_centres))
        n = np.vstack((self.unprojected_gaze_vectors[0][:, 0:2],
                       self.unprojected_gaze_vectors[1][:, 0:2]))  # [:, 0:2] takes only 2D projection

        # Normalisation of the 2D projection of gaze vectors is done inside intersect()
        if ransac == True:
            samples_to_fit = np.ceil(a.shape[0]/5).astype(np.int)  # Assuming 20% of outliners
            self.projected_eye_centre = fit_ransac(a, n, max_iters=max_iters, samples_to_fit=samples_to_fit,
                                                   min_distance=min_distance)
        else:
            self.projected_eye_centre = intersect(a, n)
        if (self.projected_eye_centre is None):
            raise TypeError("Projected_eye_centre was not fitted. You may need -v and -m argument to check whether the pupil segmentation works properly.")
        return self.projected_eye_centre

    def estimate_eye_sphere(self):
        # This function is called once after fit_projected_eye_centre()
        # self.initial_eye_z is required (in pixel unit)
        # self.initial_eye_z shall be the z-distance between the point and camera vertex (in camera frame)
        if (self.projected_eye_centre is None):
            # pdb.set_trace()
            raise TypeError('Projected_eye_centre must be initialized first')

        # Unprojecting the 2D projected eye centre to 3D.
        # Converting the projected_eye_centre from numpy indexing frame to camera frame
        projected_eye_centre_camera_frame = self.projected_eye_centre.copy()
        projected_eye_centre_camera_frame[0] = projected_eye_centre_camera_frame[0] - self.image_shape[1] / 2
        projected_eye_centre_camera_frame[1] = projected_eye_centre_camera_frame[1] - self.image_shape[0] / 2

        # Unprojection: Nearest intersection of two lines. 
        # a = [eye_centre, pupil_3Dcentre], n =[gaze_vector, pupil_3D_centre]
        projected_eye_centre_camera_frame_scaled = reverse_reproject(projected_eye_centre_camera_frame,
                                                                     self.initial_eye_z, self.focal_length)
        eye_centre_camera_frame = np.append(projected_eye_centre_camera_frame_scaled, self.initial_eye_z).reshape(3, 1)

        # Reconstructed selected gaze vectors and pupil positions by rejecting those pointing away from projected eyecentre
        m = self.unprojected_gaze_vectors[0].shape[0]
        for i in range(m):
            gazes = [self.unprojected_gaze_vectors[0][i, :].reshape(3, 1),
                     self.unprojected_gaze_vectors[1][i, :].reshape(3, 1)]
            positions = [self.unprojected_3D_pupil_positions[0][i, :].reshape(3, 1),
                         self.unprojected_3D_pupil_positions[1][i, :].reshape(3, 1)]
            selected_gaze, selected_position = self.select_pupil_from_single_observation(gazes, positions,
                                                                                         eye_centre_camera_frame)

            self.selected_gazes, self.selected_pupil_positions = self.stacking_from_nx1_to_mxn(
                [self.selected_gazes, self.selected_pupil_positions],
                [selected_gaze, selected_position],
                [3, 3])

        radius_counter = []
        for i in range(self.selected_gazes.shape[0]):
            gaze = self.selected_gazes[i, :].reshape(1, 3)
            position = self.selected_pupil_positions[i, :].reshape(1, 3)

            # Before stacking, you must reshape (3,1) to (1,3)
            a_3Dfitting = np.vstack((eye_centre_camera_frame.reshape(1, 3), position))
            n_3Dfitting = np.vstack((gaze, (position / np.linalg.norm(position))))

            intersected_pupil_3D_centre = intersect(a_3Dfitting, n_3Dfitting)
            radius = np.linalg.norm(intersected_pupil_3D_centre - eye_centre_camera_frame)
            radius_counter.append(radius)
        aver_radius = np.mean(radius_counter)

        self.aver_eye_radius = aver_radius
        self.eye_centre = eye_centre_camera_frame
        return aver_radius, radius_counter

    def gen_consistent_pupil(self):
        # This function must be called after using unproject_single_observation() to update surrent observation
        if (self.eye_centre is None) or (self.aver_eye_radius is None):
            raise TypeError("Call estimate_eye_sphere() to initialize eye_centre and eye_radius first.")
        else:
            selected_gaze, selected_position = self.select_pupil_from_single_observation(
                [self.current_gaze_pos, self.current_gaze_neg],
                [self.current_pupil_3Dcentre_pos, self.current_pupil_3Dcentre_neg], self.eye_centre)
            o = np.zeros((3, 1))

            try:
                d1, d2 = line_sphere_intersect(self.eye_centre, self.aver_eye_radius, o,
                                               selected_position / np.linalg.norm(selected_position))
                new_position_min = o + min([d1, d2]) * (selected_position / np.linalg.norm(selected_position))
                new_position_max = o + max([d1, d2]) * (selected_position / np.linalg.norm(selected_position))
                new_radius_min = (self.pupil_radius / selected_position[2, 0]) * new_position_min[2, 0]
                new_radius_max = (self.pupil_radius / selected_position[2, 0]) * new_position_max[2, 0]

                new_gaze_min = new_position_min - self.eye_centre
                new_gaze_min = new_gaze_min / np.linalg.norm(new_gaze_min)

                new_gaze_max = new_position_max - self.eye_centre
                new_gaze_max = new_gaze_max / np.linalg.norm(new_gaze_max)
                self.pupil_new_position_min, self.pupil_new_position_max = new_position_min, new_position_max
                self.pupil_new_radius_min, self.pupil_new_radius_max = new_radius_min, new_radius_max
                self.pupil_new_gaze_min, self.pupil_new_gaze_max = new_gaze_min, new_gaze_max
                consistence = True

            except(NoIntersectionError):
                # print("Cannot find line-sphere interception. Old pupil parameters are used.")
                new_position_min, new_position_max = selected_position, selected_position
                new_gaze_min, new_gaze_max = selected_gaze, selected_gaze
                new_radius_min, new_radius_max = self.pupil_radius, self.pupil_radius
                consistence = False

            return [new_position_min, new_position_max], [new_gaze_min, new_gaze_max], [new_radius_min,
                                                                                        new_radius_max], consistence

    def plot_gaze_lines(self, ax):
        t = np.linspace(-1000, 1000, 1000)
        a = np.vstack((self.ellipse_centres, self.ellipse_centres))
        n = np.vstack((self.unprojected_gaze_vectors[0][:, 0:2],
                       self.unprojected_gaze_vectors[1][:, 0:2]))  # [:, 0:2] takes only 2D projection

        for i in range(a.shape[0]):
            a_each = a[i, :]
            n_each = n[i, :]

            points = np.array(a_each).reshape(2, 1) + (t * n_each[0:2].reshape(2, 1))
            ax.plot(points[0, :], points[1, :])
        ax.set_xlim(0, self.image_shape[1])
        ax.set_ylim(self.image_shape[0], 0)
        return ax

    def select_pupil_from_single_observation(self, gazes, positions, eye_centre_camera_frame):
        # gazes is a list ~ [gaze_vector_pos~(3,1), gaze_vector_neg~(3,1)]
        # positions is a list ~ [pupil_position_pos~(3,1), pupil_position_neg~(3,1)]
        # eye_centre_camera_frame ~ numpy array~(3,1)

        selected_gaze = gazes[0]
        selected_position = positions[0]
        projected_centre = reproject(eye_centre_camera_frame, self.focal_length)
        projected_gaze = reproject(selected_position + selected_gaze, self.focal_length) - projected_centre
        projected_position = reproject(selected_position, self.focal_length)
        if np.dot(projected_gaze.T, (projected_position - projected_centre)) > 0:
            return selected_gaze, selected_position
        else:
            return gazes[1], positions[1]

    @staticmethod
    def stacking_from_nx1_to_mxn(stacked_arrays_list, stacked_vectors_list, dims_list):
        list_as_array = np.array([stacked_arrays_list])
        new_stacked_arrays_list = []
        if np.all(list_as_array == None):
            for stacked_array, stacked_vector, n in zip(stacked_arrays_list, stacked_vectors_list, dims_list):
                stacked_array = stacked_vector.reshape(1, n)
                new_stacked_arrays_list.append(stacked_array)
        elif np.all(list_as_array != None):
            for stacked_array, stacked_vector, n in zip(stacked_arrays_list, stacked_vectors_list, dims_list):
                stacked_array = np.vstack((stacked_array, stacked_vector.reshape(1, n)))
                new_stacked_arrays_list.append(stacked_array)
        elif np.any(list_as_array == None):
            print("Error list =\n", stacked_arrays_list)
            raise TypeError("Some lists are initialized, some are not ('None'). Error has happened!")
        else:
            print("Error list =\n", stacked_arrays_list)
            raise TypeError("Unknown Error Occurred.")
        return new_stacked_arrays_list

In [0]:
#inferer.py
class gaze_inferer(object):
    def __init__(self, model, flen, ori_video_shape, sensor_size, infer_gaze_flag=True):
        """
        Initialize necessary parameters and load deep_learning model
        
        Args:
            model: Deep learning model that perform image segmentation. Pre-trained model is provided at https://github.com/pydsgz/DeepVOG/model/DeepVOG_model.py, simply by loading load_DeepVOG() with "DeepVOG_weights.h5" in the same directory. If you use your own model, it should take input of grayscale image (m, 240, 320, 3) with value float [0,1] and output (m, 240, 320, 3) with value float [0,1] where (m, 240, 320, 1) is the pupil map.
            
            flen (float): Focal length of camera in mm. You can look it up at the product menu of your camera
            
            ori_video_shape (tuple or list or np.ndarray): Original video shape from your camera, (height, width) in pixel. If you cropped the video before, use the "original" shape but not the cropped shape
            
            sensor_size (tuple or list or np.ndarray): Sensor size of your camera, (height, width) in mm. For 1/3 inch CMOS sensor, it should be (3.6, 4.8). Further reference can be found in https://en.wikipedia.org/wiki/Image_sensor_format and you can look up in your camera product menu
        
        """
        # Assertion of shape
        try:
            assert ((isinstance(flen, int) or isinstance(flen, float)))
            assert (isinstance(ori_video_shape, tuple) or isinstance(ori_video_shape, list) or isinstance(
                ori_video_shape, np.ndarray))
            assert (isinstance(sensor_size, tuple) or isinstance(sensor_size, list) or isinstance(sensor_size,
                                                                                                  np.ndarray))
            assert (isinstance(sensor_size, tuple) or isinstance(sensor_size, list) or isinstance(sensor_size,
                                                                                                  np.ndarray))
        except AssertionError:
            print("At least one of your arguments does not have correct type")
            raise TypeError

        # Parameters dealing with camera and video shape
        self.flen = flen
        self.ori_video_shape, self.sensor_size = np.array(ori_video_shape).squeeze(), np.array(sensor_size).squeeze()
        self.mm2px_scaling = np.linalg.norm(self.ori_video_shape) / np.linalg.norm(self.sensor_size)

        self.model = model
        self.confidence_fitting_threshold = 0.96
        self.eyefitter = SingleEyeFitter(focal_length=self.flen * self.mm2px_scaling,
                                         pupil_radius=2 * self.mm2px_scaling,
                                         initial_eye_z=50 * self.mm2px_scaling)
        self.infer_gaze_flag = infer_gaze_flag
    def process(self, video_src, mode, output_record_path="", batch_size=32,
                output_video_path="", heatmap=False, print_prefix=""):
        """
        Parameters
        ----------
        video_src : str
            Path of the video from which you want to (1) fit the eyeball model or (2) infer the gaze.
        mode : str
            There are two modes: "Fit" or "Infer". "Fit" will fit an eyeball model from the video source.
            "Infer" will infer the gaze from the video source.
        batch_size : int
            Batch size. Recommended >= 32.
        output_record_path : str
            Path of the csv file of your gaze estimation result. Only matter if the mode == "Infer".
        output_video_path : str
            Path of the output visualization video. If mode == "Fit", it draws segmented pupil ellipse.
            If mode == "Infer", it draws segmented pupil ellipse and gaze vector. if output_video_path == "",
            no visualization will be produced.
        heatmap : bool
            If True, show heatmap in the visualization video. If False, no heatmap will be shown.
        print_prefix : str
            What to print before the progress text.
        Returns
        -------
        None
        """
        # Get video information (path strings, frame reader, video's shapes...etc)
        video_name_root, ext, vreader, vid_shapes, shape_correct, image_scaling_factor = self._get_video_info(video_src)
        (vid_m, vid_w, vid_h, vid_channels) = vid_shapes

        self.vid_manager = VideoManager(vreader=vreader, output_record_path=output_record_path,
                                        output_video_path=output_video_path, heatmap=heatmap)

        # (predict) Check if the eyeball model is imported
        if mode == "Infer":
            self._check_eyeball_model_exists()

        # Correct eyefitter's parameters in accordance with the image resizing
        self.eyefitter.focal_length = self.flen * self.mm2px_scaling * image_scaling_factor
        self.eyefitter.pupil_radius = 2 * self.mm2px_scaling * image_scaling_factor

        # Set batch-wise operation details
        initial_frame, final_frame = 0, vid_m
        final_batch_size = vid_m % batch_size
        final_batch_idx = vid_m - final_batch_size
        X_batch = np.zeros((batch_size, 240, 320, 3))
        X_batch_final = np.zeros((vid_m % batch_size, 240, 320, 3))

        # Start looping for batch-wise processing
        for idx, frame in enumerate(vreader.nextFrame()):

            print("\r%s%s %s (%d/%d)" % (print_prefix, mode, video_name_root + ext, idx + 1, vid_m), end="", flush=True)

            frame_preprocessed = self._preprocess_image(frame, shape_correct)
            mini_batch_idx = idx % batch_size

            # Before reaching the batch size, stack the array
            if ((mini_batch_idx != 0) and (idx < final_batch_idx)) or (idx == 0):
                X_batch[mini_batch_idx, :, :, :] = frame_preprocessed

            # After reaching the batch size, but not the final batch, predict heatmap and fit/infer angles
            elif (mini_batch_idx == 0) and (idx < final_batch_idx) or (idx == final_batch_idx):
                Y_batch = self.model.predict(X_batch)
                if mode == "Fit":
                    self._fitting_batch(X_batch=X_batch,
                                        Y_batch=Y_batch)
                elif mode == "Infer":
                    _, _, _ = self._infer_batch(X_batch=X_batch,
                                                Y_batch=Y_batch,
                                                idx=idx - final_batch_size)

                # Renew X_batch for next batch
                X_batch = np.zeros((batch_size, 240, 320, 3))
                X_batch[mini_batch_idx, :, :, :] = frame_preprocessed

            # Within the final batch but not yet reaching the last index, stack the array
            elif (idx > final_batch_idx) and (idx != final_frame - 1):
                X_batch_final[idx - final_batch_idx, :, :, :] = frame_preprocessed

            # Within the final batch and reaching the last index, predict heatmap and fit/infer angles
            elif idx == final_frame - 1:
                X_batch_final[idx - final_batch_idx, :, :, :] = frame_preprocessed
                Y_batch = self.model.predict(X_batch_final)
                if mode == "Fit":
                    self._fitting_batch(X_batch=X_batch_final,
                                        Y_batch=Y_batch)
                elif mode == "Infer":
                    _, _, _ = self._infer_batch(X_batch=X_batch_final,
                                                Y_batch=Y_batch,
                                                idx=idx - final_batch_size)

        if mode == "Fit":
            # Fit eyeball models. Parameters are stored as internal attributes of Eyefitter instance.
            _ = self.eyefitter.fit_projected_eye_centre(ransac=True, max_iters=100, min_distance=10*vid_m)
            _, _ = self.eyefitter.estimate_eye_sphere()

            # Issue error if eyeball model still does not exist after fitting.
            if (self.eyefitter.eye_centre is None) or (self.eyefitter.aver_eye_radius is None):
                raise TypeError("Eyeball model was not fitted. You may need -v or -m argument to check whether the pupil segmentation works properly.")
        print()

    def save_eyeball_model(self, path):
        """
        Save eyeball model parameters in json format.
        
        Args:
            path (str): path of the eyeball model file.
        """

        if (self.eyefitter.eye_centre is None) or (self.eyefitter.aver_eye_radius is None):
            print("3D eyeball model not found. You may need -v or -m argument to check whether the pupil segmentation works properly.")
            raise
        else:
            save_dict = {"eye_centre": self.eyefitter.eye_centre.tolist(),
                         "aver_eye_radius": self.eyefitter.aver_eye_radius}
            save_json(path, save_dict)

    def load_eyeball_model(self, path):
        """
        Load eyeball model parameters of json format from path.
        
        Args:
            path (str): path of the eyeball model file.
        """
        loaded_dict = load_json(path)
        if (self.eyefitter.eye_centre is not None) or (self.eyefitter.aver_eye_radius is not None):
            warnings.warn("3D eyeball exists and reloaded")

        self.eyefitter.eye_centre = np.array(loaded_dict["eye_centre"])
        self.eyefitter.aver_eye_radius = loaded_dict["aver_eye_radius"]


    def _fitting_batch(self, X_batch, Y_batch):

        if self.vid_manager.output_video_flag:
            # Convert video frames to 8 bit integer format for drawing the output video frames
            video_frames_batch = np.around(X_batch * 255).astype(int)
            vid_frame_shape_2d = (video_frames_batch.shape[1], video_frames_batch.shape[2])

        for batch_idx, (X_each, Y_each) in enumerate(zip(X_batch, Y_batch)):
            pred_each = Y_each[:, :, 1]
            _, _, _, _, ellipse_info = self.eyefitter.unproject_single_observation(pred_each)
            (rr, cc, centre, w, h, radian, ellipse_confidence) = ellipse_info

            # If visualization is true, initialize output frame
            if self.vid_manager.output_video_flag:
                vid_frame = video_frames_batch[batch_idx,]

            # Fit each observation to eyeball model
            if centre is not None:
                if (ellipse_confidence > self.confidence_fitting_threshold):
                    self.eyefitter.add_to_fitting()

                # Draw ellipse and pupil centre on input video if visualization is enabled
                if self.vid_manager.output_video_flag:
                    ellipse_centre_np = np.array(centre)

                    # Draw pupil ellipse
                    vid_frame = draw_ellipse(output_frame=vid_frame, frame_shape=vid_frame_shape_2d,
                                             ellipse_info=ellipse_info, color=[255, 255, 0])
                    # Draw small circle at the ellipse centre
                    vid_frame = draw_circle(output_frame=vid_frame, frame_shape=vid_frame_shape_2d,
                                            centre=ellipse_centre_np, radius=5, color=[0, 255, 0])

                    self.vid_manager.write_frame_with_condition(vid_frame=vid_frame, pred_each=pred_each)
            else:
                # Draw original input frame when no ellipse is found
                if self.vid_manager.output_video_flag:
                    self.vid_manager.write_frame_with_condition(vid_frame=vid_frame, pred_each=pred_each)

    def _infer_batch(self, X_batch, Y_batch, idx):

        if self.vid_manager.output_video_flag:
            # Convert video frames to 8 bit integer format for drawing the output video frames
            video_frames_batch = np.around(X_batch * 255).astype(int)
            vid_frame_shape_2d = (video_frames_batch.shape[1], video_frames_batch.shape[2])

        for batch_idx, (X_each, Y_each) in enumerate(zip(X_batch, Y_batch)):

            frame = idx + batch_idx + 1
            pred_each = Y_each[:, :, 1]
            _, _, _, _, ellipse_info = self.eyefitter.unproject_single_observation(pred_each)
            (rr, cc, centre, w, h, radian, ellipse_confidence) = ellipse_info
            # If visualization is true, initialize output frame for drawing
            if self.vid_manager.output_video_flag:
                vid_frame = video_frames_batch[batch_idx,]

            # If ellipse fitting is successful, i.e. an ellipse is located, AND gaze inference is ENABLED
            if (centre is not None) and self.infer_gaze_flag:
                p_list, n_list, _, consistence = self.eyefitter.gen_consistent_pupil()
                p1, n1 = p_list[0], n_list[0]
                px, py, pz = p1[0, 0], p1[1, 0], p1[2, 0]
                x, y = convert_vec2angle31(n1)
                positions = (px, py, pz, centre[0], centre[1])  # Pupil 3D positions and 2D projected positions
                gaze_angles = (x, y)  # horizontal and vertical gaze angles
                inference_confidence = (ellipse_confidence, consistence)
                self.vid_manager.write_results(frame_id=frame, pupil2D_x=centre[0], pupil2D_y=centre[1], gaze_x=x,
                                               gaze_y=y, confidence=ellipse_confidence, consistence=consistence)

                if self.vid_manager.output_video_flag:
                    # # Code below is for drawing video
                    ellipse_centre_np = np.array(centre)
                    projected_eye_centre = reproject(self.eyefitter.eye_centre,
                                                     self.eyefitter.focal_length)  # shape (2,1)
                    # The lines below are for translation from camera coordinate system (centred at image centre)
                    # to numpy's indexing frame. You substract the vector by the half of the video's 2D shape.
                    # Col = x-axis, Row = y-axis
                    projected_eye_centre += np.array(vid_frame_shape_2d).T.reshape(-1, 1) / 2

                    vid_frame = self._draw_vis_on_frame(vid_frame, vid_frame_shape_2d, ellipse_info, ellipse_centre_np,
                                                        projected_eye_centre, gaze_vec=n1)

                    self.vid_manager.write_frame_with_condition(vid_frame=vid_frame, pred_each=pred_each)

            # If ellipse fitting is successful, i.e. an ellipse is located, AND gaze inference is DISABLED
            elif (centre is not None) and (not self.infer_gaze_flag):
                positions, gaze_angles, inference_confidence = None, None, None
                self.vid_manager.write_results(frame_id=frame, pupil2D_x=centre[0], pupil2D_y=centre[1], gaze_x=np.nan,
                                               gaze_y=np.nan, confidence=ellipse_confidence, consistence=np.nan)
                if self.vid_manager.output_video_flag:
                    ellipse_centre_np = np.array(centre)

                    # Draw pupil ellipse
                    vid_frame = draw_ellipse(output_frame=vid_frame, frame_shape=vid_frame_shape_2d,
                                             ellipse_info=ellipse_info, color=[255, 255, 0])
                    # Draw small circle at the ellipse centre
                    vid_frame = draw_circle(output_frame=vid_frame, frame_shape=vid_frame_shape_2d,
                                            centre=ellipse_centre_np, radius=5, color=[0, 255, 0])
                    self.vid_manager.write_frame_with_condition(vid_frame=vid_frame, pred_each=pred_each)

            # IF ellipse fitting is unsuccessful.
            else:
                # If ellipse cannot be found, fill the outputs with None's
                positions, gaze_angles, inference_confidence = None, None, None

                self.vid_manager.write_results(frame_id=frame, pupil2D_x=np.nan, pupil2D_y=np.nan, gaze_x=np.nan,
                                               gaze_y=np.nan, confidence=np.nan, consistence=np.nan)

                # Draw original input frame when no ellipse is found
                if self.vid_manager.output_video_flag:
                    self.vid_manager.write_frame_with_condition(vid_frame=vid_frame, pred_each=pred_each)

        return positions, gaze_angles, inference_confidence

    def _get_video_info(self, video_src):
        video_name_with_ext = os.path.split(video_src)[1]
        video_name_root, ext = os.path.splitext(video_name_with_ext)
        vreader = skv.FFmpegReader(video_src)
        m, w, h, channels = vreader.getShape()
        image_scaling_factor = np.linalg.norm((240, 320)) / np.linalg.norm((h, w))
        shape_correct = self._inspectVideoShape(w, h)
        return video_name_root, ext, vreader, (m, w, h, channels), shape_correct, image_scaling_factor

    def _check_eyeball_model_exists(self):
        try:
            if self.infer_gaze_flag:
                assert isinstance(self.eyefitter.eye_centre, np.ndarray)
                assert self.eyefitter.eye_centre.shape == (3, 1)
                assert self.eyefitter.aver_eye_radius is not None
            else:
                pass
        except AssertionError as e:
            print(
                "3D eyeball mode is not found. Gaze inference cannot continue. Please fit/load an eyeball model first")
            raise e

    @staticmethod
    def _inspectVideoShape(w, h):
        if (w, h) == (240, 320):
            return True
        else:
            return False

    @staticmethod
    def _computeCroppedShape(ori_video_shape, crop_size):
        video = np.zeros(ori_video_shape)
        cropped = video[crop_size[0]:crop_size[1], crop_size[2], crop_size[3]]
        return cropped.shape

    @staticmethod
    def _preprocess_image(img, shape_correct):
        """
        
        Args:
            img (numpy array): unprocessed image with shape (w, h, 3) and values int [0, 255]
        Returns:
            output_img (numpy array): processed grayscale image with shape ( 240, 320, 1) and values float [0,1]
        """
        output_img = np.zeros((240, 320, 3))
        img = img / 255
        img = rgb2gray(img)
        if not shape_correct:
            img = resize(img, (240, 320))
        output_img[:, :, :] = img.reshape(240, 320, 1)
        return output_img

    @staticmethod
    def _draw_vis_on_frame(vid_frame, vid_frame_shape_2d, ellipse_info, ellipse_centre_np, projected_eye_centre,
                           gaze_vec):

        # Draw pupil ellipse
        vid_frame = draw_ellipse(output_frame=vid_frame, frame_shape=vid_frame_shape_2d,
                                 ellipse_info=ellipse_info, color=[255, 255, 0])

        # Draw from eyeball centre to ellipse centre (just connecting two points)
        vec_with_length = ellipse_centre_np - projected_eye_centre.squeeze()
        vid_frame = draw_line(output_frame=vid_frame, frame_shape=vid_frame_shape_2d, o=projected_eye_centre,
                              l=vec_with_length, color=[0, 0, 255])

        # Draw gaze vector originated from ellipse centre
        vid_frame = draw_line(output_frame=vid_frame, frame_shape=vid_frame_shape_2d, o=ellipse_centre_np,
                              l=gaze_vec * 50, color=[255, 0, 0])

        # Draw small circle at the ellipse centre
        vid_frame = draw_circle(output_frame=vid_frame, frame_shape=vid_frame_shape_2d,
                                centre=ellipse_centre_np, radius=5, color=[0, 255, 0])
        return vid_frame

In [0]:
#intersection.py
class NoIntersectionError(Exception):
    pass

# vector (m,n) , m = number of examples, n = dimensionality
# a = coordinates of the vector
# n = orientation of the vector
def intersect(a, n):
    # default normalisation of vectors n
    n = n/np.linalg.norm(n, axis = 1, keepdims=True)
    num_lines = a.shape[0]
    dim = a.shape[1]
    I = np.eye(dim)
    R_sum = 0
    q_sum = 0
    for i in range(num_lines):    
        R = I - np.matmul(n[i].reshape(dim,1), n[i].reshape(1,dim))

        q = np.matmul(R,a[i].reshape(dim,1))
        q_sum = q_sum + q
        R_sum = R_sum + R
    p = np.matmul(np.linalg.inv(R_sum),q_sum)
    return p

def calc_distance(a,n, p):
    num_lines = a.shape[0]
    dim = a.shape[1]
    I = np.eye(dim)
    D_sum = 0
    for i in range(num_lines):
        D_1 = (a[i].reshape(dim,1) - p.reshape(dim,1)).T
        D_2 = I - np.matmul(n[i].reshape(dim,1), n[i].reshape(1,dim))
        D_3 = D_1.T
        D = np.matmul(np.matmul(D_1,D_2),D_3)
        D_sum = D_sum + D
    D_sum = D_sum/num_lines
    return D_sum

def fit_ransac(a,n, max_iters = 2000, samples_to_fit = 20, min_distance = 2000):
    num_lines = a.shape[0]
    
    best_model = None
    best_distance = min_distance
    for i in range(max_iters):
        # print("\rRANSAC: Currently {0}".format(i), flush=True)
        sampling_index = np.random.choice(num_lines, size = samples_to_fit, replace=False)
        a_sampled = a[sampling_index,:]
        n_sampled = n[sampling_index,:]
        model_sampled = intersect(a_sampled, n_sampled)
        sampled_distance = calc_distance( a,n, model_sampled)
        # print(sampled_distance)
        if sampled_distance > min_distance:
            continue
        else:
            if sampled_distance < best_distance:
                best_model = model_sampled
                best_distance = sampled_distance
    # if best_model is None:
    #     best_model = model_sampled
    return best_model

def line_sphere_intersect(c, r, o, l):
    # c = numpy array (3,1). Centre of the eyeball
    # r = scaler. Radius of the eyeball
    # o = numpy array (3,1). Origin of the line
    # l = numpy array (3,1). Directional unit vector of the line
    # return [d1, d2] : auxilary variables of the parametrised line x = o + dl
    # the closer one to the camera is chosen
    l = l/np.linalg.norm(l)
    delta = np.square(np.dot(l.T,(o-c))) - np.dot((o-c).T,(o-c)) + np.square(r)
    if delta < 0:
        raise NoIntersectionError
    else:
        d1 = -np.dot(l.T,(o-c)) + np.sqrt(delta)
        d2 = -np.dot(l.T,(o-c)) - np.sqrt(delta)
    return [d1,d2]

In [0]:
#jobman.py
class deepvog_jobman_CLI(object):
    def __init__(self, gpu_num, flen, ori_video_shape, sensor_size, batch_size):
        """
        
        Args:
            gpu_num (str)
            flen (float)
            ori_video_shape (tuple)
            sensor_size (tuple)
            batch_size (int)
        
        """
        os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
        os.environ["CUDA_VISIBLE_DEVICES"] = gpu_num
        self.model = load_DeepVOG()
        self.flen = float(flen)
        self.ori_video_shape = ori_video_shape
        self.sensor_size = sensor_size
        self.batch_size = batch_size

    def fit(self, vid_path, output_json_path, output_video_path="", heatmap=False,
            print_prefix=""):

        inferer = gaze_inferer(self.model, self.flen, self.ori_video_shape, self.sensor_size)
        inferer.process(video_src=vid_path, mode="Fit", batch_size=self.batch_size, output_video_path=output_video_path,
                        heatmap=heatmap, print_prefix=print_prefix)
        inferer.save_eyeball_model(output_json_path)

    def infer(self, vid_path, eyeball_model_path, output_record_path, output_video_path="", heatmap=False,
              infer_gaze_flag=True, print_prefix=""):
        if isinstance(infer_gaze_flag, str):
            try:
                infer_gaze_flag = int(infer_gaze_flag)
            except ValueError:
                infer_gaze_flag = False

        inferer = gaze_inferer(self.model, self.flen, self.ori_video_shape, self.sensor_size,
                               infer_gaze_flag=infer_gaze_flag)
        if infer_gaze_flag:
            inferer.load_eyeball_model(eyeball_model_path)
        inferer.process(video_src=vid_path, mode="Infer", batch_size=self.batch_size,
                        output_record_path=output_record_path, output_video_path=output_video_path, heatmap=heatmap,
                        print_prefix=print_prefix)


class deepvog_jobman_table_CLI(deepvog_jobman_CLI):
    def __init__(self, csv_path, gpu_num, flen, ori_video_shape, sensor_size, batch_size,
                 skip_errors=False, skip_existed=False, error_log_path=""):
        self.csv_dict = csv_reader(csv_path)
        self.skip_errors, self.skip_existed, self.error_log_path = skip_errors, skip_existed, error_log_path
        super(deepvog_jobman_table_CLI, self).__init__(gpu_num, flen, ori_video_shape, sensor_size, batch_size)
        self._initialize_error_log()

    def run_batch(self):
        num_operations = len(self.csv_dict['operation'])
        operation_counts = dict()
        for i in range(num_operations):
            current_operation = self.csv_dict['operation'][i]
            operation_counts[current_operation] = operation_counts.get(current_operation, 0) + 1

        print("Total number of operations = %d" % (num_operations))
        print("     - Fit    %d/%d " % (operation_counts.get("fit", 0), num_operations))
        print("     - Infer  %d/%d " % (operation_counts.get("infer", 0), num_operations))
        print("     - Both   %d/%d " % (operation_counts.get("both", 0), num_operations))
        for i in range(num_operations):
            try:
                current_operation = self.csv_dict['operation'][i]
                progress = '%d/%d ' % (i + 1, num_operations)

                # Skip file if the output already existed
                if self.skip_existed:
                    output_existed, concerned_path = self._check_if_output_existed(current_operation, i)
                    if output_existed:
                        print("Video %d skipped (operation %s),\nbecause %s is/are found" % (i+1,
                                                                                             current_operation,
                                                                                             str(concerned_path)))
                        continue

                # Actions for each operation type
                if current_operation == "fit":
                    self.fit(vid_path=self.csv_dict['fit_vid'][i],
                             output_json_path=self.csv_dict['eyeball_model'][i],
                             print_prefix=progress)
                elif current_operation == "infer":
                    self.infer(vid_path=self.csv_dict['infer_vid'][i],
                               eyeball_model_path=self.csv_dict['eyeball_model'][i],
                               output_record_path=self.csv_dict['result'][i],
                               infer_gaze_flag=self.csv_dict["with_gaze"][i],
                               print_prefix=progress)
                elif current_operation == "both":
                    self.fit(vid_path=self.csv_dict['fit_vid'][i],
                             output_json_path=self.csv_dict['eyeball_model'][i],
                             print_prefix=progress)
                    self.infer(vid_path=self.csv_dict['infer_vid'][i],
                               eyeball_model_path=self.csv_dict['eyeball_model'][i],
                               output_record_path=self.csv_dict['result'][i],
                               infer_gaze_flag=self.csv_dict["with_gaze"][i],
                               print_prefix=progress)
            except Exception:
                existed, paths = self._check_if_output_existed(current_operation, i)
                if existed:
                    for output_path in paths:
                        os.remove(output_path)
                if self.skip_errors:
                    try:
                        print("\nError encountered. Video {} skipped.".format(i + 1))
                        if self.error_log_path:
                            self._log_error(i)
                        continue
                    except Exception:
                        print("Error encountered when logging the error.")
                        traceback.print_exc()
                        continue
                else:
                    traceback.print_exc()
                    raise

    def _initialize_error_log(self):
        if self.error_log_path:
            # This function mainly serves to create and overwrite existing logs
            # (plus add some meta information for inspection)
            with open(self.error_log_path, "w") as fh:
                opening_msg = "DEEPVOG ERROR LOG\n"
                content = "Configurations:\nFLEN:{}\nOriginal video shape:{}\nSensor size:{}\nBatch size:{}\n".format(
                    self.flen,
                    self.ori_video_shape,
                    self.sensor_size,
                    self.batch_size
                )
                fh.write(opening_msg + content)

    def _log_error(self, i):

        with open(self.error_log_path, "a") as fh:
            msg = ["{}: {}(delimiter)".format(k, v[i]) for k, v in self.csv_dict.items()]
            msg_pretty = "".join(msg).replace("(delimiter)", "\n")
            line_separator = "\n" + "="*30 + "\n"
            fh.write(line_separator + msg_pretty)
            traceback.print_exc(file=fh)
        print("Error logged in {}".format(self.error_log_path))

    def _check_if_output_existed(self, current_operation, vid_idx):
        output_existed = False
        concerned_path = ""
        if current_operation == "fit":
            # For "fit" mode, skip if eyeball model already exists
            eyeball_model_path = self.csv_dict['eyeball_model'][vid_idx]
            output_existed = os.path.isfile(eyeball_model_path)
            concerned_path = (eyeball_model_path, )
        elif current_operation == "infer":
            # For "infer" mode, skip if gaze record already exists
            output_record_path = self.csv_dict['result'][vid_idx]
            output_existed = os.path.isfile(output_record_path)
            concerned_path = (output_record_path, )
        elif current_operation == "both":
            # For "both" mode, skip if both eyeball AND gaze record already exist
            eyeball_model_path = self.csv_dict['eyeball_model'][vid_idx]
            output_record_path = self.csv_dict['result'][vid_idx]
            eyeball_existed = os.path.isfile(eyeball_model_path)
            record_existed = os.path.isfile(output_record_path)
            if eyeball_existed and record_existed:
                output_existed = True
                concerned_path = (eyeball_model_path, output_record_path)
        return output_existed, concerned_path

In [0]:
#unprojection.py
# The whole unprojection algorithm is invented by Safaee-Rad et al. 1992, see https://ieeexplore.ieee.org/document/163786
# This python script is a re-implementation of Safaee-Rad et al.'s works


def gen_cone_co(alpha, beta, gamma, a_prime, h_prime, b_prime, g_prime, f_prime, d_prime):
    gamma_square = np.power(gamma,2)
    a = gamma_square * a_prime
    b = gamma_square * b_prime
    c = a_prime * np.power(alpha,2) + 2 * h_prime * alpha * beta + b_prime * np.power(beta,2) + 2 * g_prime * alpha + 2 * f_prime * beta + d_prime
    d = gamma_square * d_prime
    f = -gamma * (b_prime * beta + h_prime * alpha +f_prime)
    g = -gamma * (h_prime * beta + a_prime * alpha + g_prime)
    h = gamma_square * h_prime
    u = gamma_square * g_prime
    v = gamma_square * f_prime
    w = -gamma * (f_prime * beta + g_prime * alpha + d_prime)
    return a,b,c,d,f,g,h,u,v,w

'''
Safaee-Rad, 1992 (8)
'''
def gen_rotmat_co(lamb, a,b,g,f,h):
    t1 = (b-lamb)*g - f*h
    t2 = (a - lamb)*f - g*h
    t3 = -(a-lamb)*(t1/t2)/g - (h/g)
    m = 1/(np.sqrt(1+np.power((t1/t2),2)+np.power(t3,2)))
    l = (t1/t2)*m
    n = t3*m
    return l, m, n
'''
Safaee-Rad, 1992 (12), (27)-(33)
'''
def gen_lmn(lamb1, lamb2, lamb3):
    if lamb1 < lamb2:
        l = 0
        m_pos = np.sqrt((lamb2-lamb1)/(lamb2-lamb3))
        m_neg = -m_pos
        n = np.sqrt((lamb1-lamb3)/(lamb2-lamb3))
        return [l, l], [m_pos, m_neg], [n, n]
    elif lamb1 > lamb2:
        l_pos = np.sqrt((lamb1-lamb2)/(lamb1-lamb3))
        l_neg = -l_pos
        n = np.sqrt((lamb2-lamb3)/(lamb1-lamb3))
        m = 0
        return [l_pos, l_neg], [m, m], [n, n]
    elif lamb1 == lamb2:
        n = 1
        m = 0
        l = 0
        return [l,l], [m,m], [n,n]
    else:
        
        logging.warning("Failure to generate l,m,n. None's are returned")
        return None, None, None
def calT3(l, m ,n ):
    lm_sqrt = np.sqrt((l**2)+(m**2))
    T3 = np.array([-m/lm_sqrt, -(l*n)/lm_sqrt, l, 0,
                       l/lm_sqrt, -(m*n)/lm_sqrt, m, 0,
                       0, lm_sqrt, n, 0,
                       0, 0, 0, 1]).reshape(4,4)
    return T3
def calABCD(T3, lamb1, lamb2, lamb3):
    li, mi, ni = T3[0:3,0], T3[0:3,1], T3[0:3,2]
    lamb_array = np.array([lamb1, lamb2, lamb3])
    A = np.dot(np.power(li,2), lamb_array)
    B = np.sum(li*ni*lamb_array)
    C = np.sum(mi*ni*lamb_array)
    D = np.dot(np.power(ni,2), lamb_array)
    return A,B,C,D
def calXYZ_perfect(A,B,C,D, r):
    
    Z = (A*r)/np.sqrt((B**2)+(C**2)-A*D)
    X = (-B/A)*Z
    Y = (-C/A)*Z
    center = np.array([X,Y,Z,1]).reshape(4,1)
    return center
    
    
    
def check_parallel(v1, v2):
    a = np.dot(v1.T, v2)
    b = np.linalg.norm(v1) * np.linalg.norm(v2)
    radian = np.arccos(a/b).squeeze()
    return np.rad2deg(radian)
def convert_ell_to_general(xc,yc, w,h,radian):
    A = (w**2)*(np.sin(radian)**2) + (h**2) * (np.cos(radian)**2)
    B = 2 * ((h**2) - (w**2)) * np.sin(radian) * np.cos(radian)
    C = (w**2)*(np.cos(radian)**2) + (h**2)*(np.sin(radian)**2)
    D = -2*A*xc - B*yc
    E = -B*xc - 2*C*yc
    F = A*(xc**2) + B*xc*yc + C*(yc**2) - (w**2)*(h**2)
    return A,B,C,D,E,F


def unprojectGazePositions(vertex, ell_co, radius = None):
    """
    This function generates (1)directions of unprojected pupil disk (gaze vector) 
    and (2) position of the pupil disk, with an assumed radius of the pupil disk
    Args:
        vectex (list or tuple): list with 3 elements of x, y, z coordinates of the camera with respect to the image frame
        ell_co (list or tuple): list of 6 coefficients of a generalised/expanded ellipse equations at the image frame
            A*(x**2) + B*x*y + C*(y**2) + D*x + E*y + F = 0 (from https://en.wikipedia.org/wiki/Ellipse#General_ellipse)
        
    Returns:
        Positive Norm of pupil disk from camera frame
        Negative Norm of pupil disk from camera frame
        Positive Norm of pupil disk from canonical frame
        negative Norm of pupil disk from canonical frame
    """
    
    # Coefficients of the general ellipse equation
    A,B,C,D,E,F = [x for x in ell_co]
    # Vertex (Point of the camera)
    alpha, beta, gamma = [x for x in vertex]
    
    # Ellipse parameter at image frame (z_c = +20) with respect to the camera frame
    a_prime = A
    h_prime = B/2
    b_prime = C
    g_prime = D/2
    f_prime = E/2
    d_prime = F
    # Coefficients of the Cone at the image frame
    a,b,c,d,f,g,h,u,v,w = gen_cone_co(alpha, beta, gamma, a_prime, h_prime, b_prime, g_prime, f_prime, d_prime)
    # Safaee-Rad, 1992 (10)
    lamb_co1 = 1
    lamb_co2 = -(a+b+c)
    lamb_co3 = (b*c + c*a + a*b - np.power(f,2) - np.power(g,2) - np.power(h,2))
    lamb_co4 = -(a*b*c + 2*f*g*h - a*np.power(f,2) - b*np.power(g,2) - c*np.power(h,2))
    lamb1, lamb2, lamb3 = np.roots([lamb_co1, lamb_co2, lamb_co3, lamb_co4])
    # generate Normal vector at the canonical frame
    
    l, m, n = gen_lmn(lamb1,lamb2,lamb3)
    norm_cano_pos = np.array([l[0],m[0],n[0],1]).reshape(4,1)
    norm_cano_neg = np.array([l[1],m[1],n[1],1]).reshape(4,1)
    
    # T1 Rotational Transformation to the camera fream
    l1, m1, n1 = gen_rotmat_co(lamb1, a,b,g,f,h)
    l2, m2, n2 = gen_rotmat_co(lamb2, a,b,g,f,h)
    l3, m3, n3 = gen_rotmat_co(lamb3, a,b,g,f,h)
    T1 = np.array([l1,l2,l3,0 ,m1, m2, m3,0, n1, n2, n3,0, 0,0,0,1]).reshape(4,4)
    li, mi, ni = T1[0,0:3], T1[1,0:3], T1[2,0:3]
    if np.cross(li,mi).dot(ni) < 0:
        li = -li
        mi = -mi
        ni = -ni
    T1[0,0:3], T1[1,0:3], T1[2,0:3] = li, mi, ni
    norm_cam_pos = np.dot(T1,  norm_cano_pos)
    norm_cam_neg = np.dot(T1, norm_cano_neg)

    # Calculating T2
    T2 = np.eye(4)
    T2[0:3,3] = -(u*li+v*mi+w*ni)/np.array([lamb1, lamb2, lamb3])
    # Calculating T3
    T3_pos = calT3(l[0], m[0], n[0])
    T3_neg = calT3(l[1], m[1], n[1])
    # calculate ABCD
    A_pos, B_pos, C_pos, D_pos = calABCD(T3_pos, lamb1, lamb2, lamb3)
    A_neg, B_neg, C_neg, D_neg = calABCD(T3_neg, lamb1, lamb2, lamb3)
    # Calculating T0
    T0 = np.eye(4)

    T0[2,3] = -gamma # -gamma = -(vertex[2]) = -(-focal_length) = + focal_length
    # Calculating center position with respect to the perfect frame
    center_pos = calXYZ_perfect(A_pos, B_pos, C_pos, D_pos, radius)
    center_neg = calXYZ_perfect(A_neg, B_neg, C_neg, D_neg, radius)
    # From perfect frame to camera frame
    true_center_pos = np.matmul(T0,np.matmul(T1,np.matmul(T2,np.matmul(T3_pos,center_pos))))
    if true_center_pos[2] <0:
        center_pos[0:3] = -center_pos[0:3]
        true_center_pos = np.matmul(T0,np.matmul(T1,np.matmul(T2,np.matmul(T3_pos,center_pos))))
    true_center_neg = np.matmul(T0,np.matmul(T1,np.matmul(T2,np.matmul(T3_neg,center_neg))))
    if true_center_neg[2] <0:
        center_neg[0:3] = -center_neg[0:3]
        true_center_neg = np.matmul(T0,np.matmul(T1,np.matmul(T2,np.matmul(T3_neg,center_neg))))

    return norm_cam_pos[0:3], norm_cam_neg[0:3], true_center_pos[0:3], true_center_neg[0:3]

def reproject(vec_3d, focal_length, batch_mode= False):
    # vec_3d = (3,1) numpy array: Coordinates of the 3D unprojected object in CAMERA frame
    # vec_3d can also be (3,), but not (1,3)
    if batch_mode == False:
        vec_2d = (focal_length*vec_3d[0:2])/vec_3d[2]
        
    else:
        # converting vec_3d ~ (m,3) to vec_2d~(m,2)
        vec_2d = (focal_length*(vec_3d[:,0:2]))/vec_3d[:,[2]]
    return vec_2d
    
def reverse_reproject(vec_2d, z, focal_length):
    # Scale the x,y in a reverse manner of reproject() function,
    # when you unproject the reprojected coordinate.
    vec_2d_scaled = (vec_2d*z)/focal_length
    return vec_2d_scaled

In [0]:
#utils.py
def convert_vec2angle31(n1):
    """
    Inputs:
        n1 = numpy array with shape (3,1)
    """
    assert n1.shape == (3,1)
    n1 = n1/np.linalg.norm(n1)
    n1_x, n1_y, n1_z_abs = n1[0,0], n1[1,0], np.abs(n1[2,0])
    # x-augulation            
    if n1_x > 0:
        x_angle = np.arctan(n1_z_abs/n1_x)
    else:
        x_angle = np.pi - np.arctan(n1_z_abs/np.abs(n1_x))
    # y-angulation
    if n1_y > 0:
        y_angle = np.arctan(n1_z_abs/n1_y)
    else:
        y_angle = np.pi - np.arctan(n1_z_abs/np.abs(n1_y))
    x_angle = np.rad2deg(x_angle)
    y_angle = np.rad2deg(y_angle)
    return [x_angle, y_angle]

def save_json(path, save_dict):
    json_str = json.dumps(save_dict, indent=4)
    with open(path, "w") as fh:
        fh.write(json_str)
        
def load_json(path):
    with open(path, "r+") as fh:
        json_str = fh.read()
    return json.loads(json_str)

def csv_reader(csv_path):
    col_dict = dict()
    col_list = []
    with open(csv_path, "r") as fh:
        for idx, line in enumerate(fh):
            row = line.split(",")
            row_stripped = list(map(lambda x : x.strip(), row))
            if idx == 0:
                for col in row_stripped:
                    col_list.append(col)
                    col_dict[str(col)] = []
            else:
                for col_idx, col in enumerate(row_stripped):
                    col_dict[col_list[col_idx]].append(str(col))
    return col_dict

In [0]:
#visualization.py
def draw_line(output_frame, frame_shape, o, l, color=[255, 0, 0]):
    """
    Parameters
    ----------
    output_frame : numpy.darray
        Video frame to draw the circle. The value of video frame should be of type int [0, 255]
    frame_shape : list or tuple or numpy.darray
        Shape of the frame. For example, (240, 320)
    o : list or tuple or numpy.darray
        Origin of the line, with shape (2,) denoting (x, y).
    l : list or tuple or numpy.darray
        Vector with length. Body of the line. Shape = (2, ), denoting (x, y)
    color : tuple or list or numpy.darray
        RBG colors, e.g. [255, 0, 0] (red color), values of type int [0, 255]
    Returns
    -------
    output frame : numpy.darray
        Frame with the ellipse drawn.
    """
    R, G, B = color
    rr, cc = line(int(np.round(o[0])), int(np.round(o[1])), int(np.round(o[0] + l[0])), int(np.round(o[1] + l[1])))
    rr[rr > int(frame_shape[1]) - 1] = frame_shape[1] - 1
    cc[cc > int(frame_shape[0]) - 1] = frame_shape[0] - 1
    rr[rr < 0] = 0
    cc[cc < 0] = 0
    output_frame[cc, rr, 0] = R
    output_frame[cc, rr, 1] = G
    output_frame[cc, rr, 2] = B
    return output_frame


def draw_ellipse(output_frame, frame_shape, ellipse_info, color=[255, 255, 0]):
    """
    Draw a circle on an image or video frame. Drawing will be discretized.
    Parameters
    ----------
    output_frame : numpy.darray
        Video frame to draw the circle. The value of video frame should be of type int [0, 255]
    frame_shape : list or tuple or numpy.darray
        Shape of the frame. For example, (240, 320)
    ellipse_info : list or tuple
        Information of ellipse parameters. (rr, cc, centre, w, h, radian, ellipse_confidence).
    color : tuple or list or numpy.darray
        RBG colors, e.g. [255, 0, 0] (red color), values of type int [0, 255]
    Returns
    -------
    output frame : numpy.darray
        Frame withe the ellipse drawn.
    """

    R, G, B = color
    (rr, cc, centre, w, h, radian, ellipse_confidence) = ellipse_info
    rr[rr > int(frame_shape[1]) - 1] = frame_shape[1] - 1
    cc[cc > int(frame_shape[0]) - 1] = frame_shape[0] - 1
    rr[rr < 0] = 0
    cc[cc < 0] = 0
    output_frame[cc, rr, 0] = R
    output_frame[cc, rr, 1] = G
    output_frame[cc, rr, 2] = B
    return output_frame


def draw_circle(output_frame, frame_shape, centre, radius, color=[255, 0, 0]):
    """
    Draw a circle on an image or video frame. Drawing will be discretized.
    Parameters
    ----------
    output_frame : numpy.darray
        Video frame to draw the circle. The value of video frame should be of type int [0, 255]
    frame_shape : list or tuple or numpy.darray
        Shape of the frame. For example, (240, 320)
    centre : list or tuple or numpy.darray
        x,y coordinate of the circle centre
    radius : int or float
        Radius of the circle to draw.
    color : tuple or list or numpy.darray
        RBG colors, e.g. [255, 0, 0] (red color), values of type int [0, 255]
    Returns
    -------
    output frame : numpy.darray
        Frame withe the circle drawn.
    """

    R, G, B = color
    rr_p1, cc_p1 = circle_perimeter(int(np.round(centre[0])), int(np.round(centre[1])), radius)
    rr_p1[rr_p1 > int(frame_shape[1]) - 1] = frame_shape[1] - 1
    cc_p1[cc_p1 > int(frame_shape[0]) - 1] = frame_shape[0] - 1
    rr_p1[rr_p1 < 0] = 0
    cc_p1[cc_p1 < 0] = 0
    output_frame[cc_p1, rr_p1, 0] = R
    output_frame[cc_p1, rr_p1, 1] = G
    output_frame[cc_p1, rr_p1, 2] = B
    return output_frame


class VideoManager:
    def __init__(self, vreader, output_record_path="", output_video_path="", heatmap=False):
        # Parameters
        self.vreader = vreader
        self.heatmap = heatmap
        self.output_video_flag = True if output_video_path else False
        self.output_record_flag = True if output_record_path else False
        self.vwriter = skv.FFmpegWriter(output_video_path) if self.output_video_flag else None
        self.results_recorder = open(output_record_path, "w") if self.output_record_flag else None

        # Initialization actions
        self._initialize_results_recorder()

    def write_frame_with_condition(self, vid_frame, pred_each):

        if self.heatmap:
            heatmap_frame = np.zeros((pred_each.shape[0], pred_each.shape[1], 3))  # Shape = (w, h, 3)
            heatmap_frame[:, :, :] = np.around(
                pred_each.reshape(pred_each.shape[0], pred_each.shape[1], 1) * 255).astype(int)
            output_frame = np.concatenate((vid_frame, heatmap_frame), axis=1)
            self.vwriter.writeFrame(output_frame)
        else:
            self.vwriter.writeFrame(vid_frame)

    def write_results(self, frame_id, pupil2D_x, pupil2D_y, gaze_x, gaze_y, confidence, consistence):
        self.results_recorder.write("%d,%f,%f,%f,%f,%f,%f\n" % (frame_id, pupil2D_x, pupil2D_y,
                                                                gaze_x, gaze_y,
                                                                confidence, consistence))

    def _initialize_results_recorder(self):
        if self.output_record_flag:
            self.results_recorder.write("frame,pupil2D_x,pupil2D_y,gaze_x,gaze_y,confidence,consistence\n")

    def __del__(self):
        self.vreader.close()
        if self.vwriter:
            self.vwriter.close()
        if self.results_recorder:
            self.results_recorder.close()

#main.py

In [22]:
import argparse
from argparse import RawTextHelpFormatter
from ast import literal_eval

# Running DeepVOG with command-line parser.

description_text = """
Belows are the examples of usage. Don't forget to set up camera parameters such as focal length, because it varies from equipment to equipment and is necessaray for accuracy.
    Example #1 - Fitting an eyeball model (using default camera parameters)
    python -m deepvog --fit ~/video01.mp4 ~/model01.json
    Example #2 - Infer gaze (using default camera parameters)
    python -m deepvog --infer ~/video01.mp4 ~/model01.json ~/model1_video01.csv
    Example #3 - Setting up necessary parameters (focal length = 12mm, video shape = (240,320), ... etc)
    python -m deepvog --fit ./vid.mp4 ./model.json -f 12 -vs 240 320 -s 3.6 4.8 -b 32 -g 0
Caution: If your video has an aspect ratio (height/width) other than 0.75, please crop it manually until it reaches the required aspect ratio otherwise the gaze estimate will not be accurate. Futher release will include cropping option.
    Example #4 - If you cropped the video from (250,390) to (240,360), keep using the argument "-vs 250 390"
    python -m deepvog --fit ./vid.mp4 ./model.json -f 12 -vs 250 390 -s 3.6 4.8 -b 32 -g 0
    
You can also fit and infer a video from a csv table:
    Example #5
    python -m deepvog --table ./table.csv -f 12 -vs 240 320 -s 3.6 4.8 -b 32 -g 0
The csv file must follow the format below:
    
    operation, fit_vid,             infer_vid,              eyeball_model,      result
    fit      , /PATH/fit_vid.mp4,   /PATH/infer_vid.mp4,    /PATH/model.json,   /PATH/output_result.csv
    infer    , /PATH/fit_vid2.mp4,  /PATH/infer_vid2.mp4,   /PATH/model2.json,  /PATH/output_result2.csv
    both     , /PATH/fit_vid3.mp4,  /PATH/infer_vid3.mp4,   /PATH/model3.json,  /PATH/output_result3.csv
    fit      , /PATH/fit_vid4.mp4,  /PATH/infer_vid4.mp4,   /PATH/model4.json,  /PATH/output_result4.csv
    ...
The "operation" column should contain either fit/infer/both: 
    1. fit: it will fit the video specified by the path "fit_vid" and save the model to the path specified by "eyeball_model". Other columns are irrelevant and can be omited.
    2. infer: it will load the model from the path specified in "eyeball_model", and infer the video from the path specified in "infer_vid", and then save the result to the path specified by "result".
    3. both: first performs "fit", then performs "infer".
"""
'''
fit_help = "Fitting an eyeball model. Call with --fit [video_src_path] [eyeball_model_saving_path]."
infer_help = "Inter video from eyeball model. Call with --infer [video_scr_path] [eyeball_model_path] [results_saving_path]"
table_help = "Fit or infer videos from a csv table. The column names of the csv must follow a format (see --help description). Call with --table [csv_path]"
ori_vid_shape_help = "Original and uncropped video shape of your camera output, height and width in pixel. Default = 240 320"
flen_help = "Focal length of your camera in mm."
gpu_help = "GPU device number. Default = 0"
sensor_help = "Sensor size of your camera digital sensor, height and width in mm. Default = 3.6 4.8"
batch_help = "Batch size for forward inference. Default = 512."
visualize_help = "Draw the visualization of ellipse fitting and gaze vector. Call with --visualize [video_output_path]. (Not yet available with --table mode)"
heatmap_help = "Show network's output of segmented pupil heatmap in visualization. Call with --heatmap. (Not yet available with --table mode)"

parser = argparse.ArgumentParser(description=description_text, formatter_class=RawTextHelpFormatter)
required = parser.add_argument_group("required arguments")
required.add_argument("--fit", help=fit_help, nargs=2, type=str, metavar=("PATH", "PATH"))
required.add_argument("--infer", help=infer_help, nargs=3, type=str, metavar=("PATH", "PATH", "PATH"))
required.add_argument("--table", help=table_help, type=str, metavar=("PATH"))
parser.add_argument("-f", "--flen", help=flen_help, default=6, type=float, metavar=("FLOAT"))
parser.add_argument("-g", "--gpu", help=gpu_help, default="0", type=str, metavar=("INT"))
parser.add_argument("-vs", "--vidshape", help=ori_vid_shape_help, default="(240, 320)", type=str, metavar=("INT,INT"))
parser.add_argument("-s", "--sensor", help=sensor_help, default="(3.6, 4.8)", type=str, metavar=("FLOAT,FLOAT"))
parser.add_argument("-b", "--batchsize", help=batch_help, default=512, type=int, metavar=("INT"))
parser.add_argument("-v", "--visualize", help=visualize_help, default="", type=str, metavar=("PATH"))
parser.add_argument("-m", "--heatmap", help=heatmap_help, default=False, action="store_true")
parser.add_argument("--skip_existed", default=False, action="store_true")
parser.add_argument("--skip_errors", default=False, action="store_true")
parser.add_argument("--log_errors", type=str, default="", metavar=("PATH"))
parser.add_argument("--no_gaze", default=True, action="store_false")
args = parser.parse_args()
'''

# Check there is EXACTLY one argument from --fit, --infer and --table
all_modes_list = [args.fit, args.infer, args.table]
cli_modes_list = all_modes_list[0:3]
num_modes = sum([x is not None for x in all_modes_list])
if num_modes != 1:
    parser.error("Exactly one argument from --fit, --infer and --table is requried")
else:

    # Command line mode
    if sum([x is not None for x in cli_modes_list]) > 0:

        from .jobman import deepvog_jobman_CLI, deepvog_jobman_table_CLI

        flen = args.flen
        gpu = args.gpu
        ori_video_shape = literal_eval(args.vidshape)
        sensor_size = literal_eval(args.sensor)
        batch_size = args.batchsize

        # Table mode
        if args.table is not None:
            jobman_table = deepvog_jobman_table_CLI(args.table, gpu, flen,
                                                    ori_video_shape, sensor_size, batch_size,
                                                    skip_errors=args.skip_errors,
                                                    skip_existed=args.skip_existed,
                                                    error_log_path=args.log_errors)
            jobman_table.run_batch()

        # Fit or Infer mode
        jobman = deepvog_jobman_CLI(gpu, flen, ori_video_shape, sensor_size, batch_size)
        if args.fit is not None:
            vid_src_fitting, eyemodel_save = args.fit
            jobman.fit(vid_path=vid_src_fitting, output_json_path=eyemodel_save, output_video_path=args.visualize,
                       heatmap=args.heatmap)
        if args.infer is not None:
            vid_scr_inference, eyemodel_load, result_output = args.infer
            jobman.infer(vid_path=vid_scr_inference,
                         eyeball_model_path=eyemodel_load,
                         output_record_path=result_output,
                         output_video_path=args.visualize,
                         heatmap=args.heatmap,
                         infer_gaze_flag=args.no_gaze)

NameError: ignored